In [2]:
def getUserRatingData(userId,MoviesRatingDataSet):
    NewUserDataSetll = MoviesRatingDataSet['userId'] == userId
    columns = ['title','rating']
    NewUserDataSet = MoviesRatingDataSet[NewUserDataSetll][columns]
    NewUserDataSet.reset_index(inplace=True ,drop=True) 
    return NewUserDataSet
def CFRecommender(AllUsersRatingDataSet,userId,MinUsersRating=100) : 
    AllUsersRating = AllUsersRatingDataSet.pivot_table(index=['userId'],columns=['title'],values='rating')
    NewUserRating = getUserRatingData(userId,AllUsersRatingDataSet)
    MoviesCorrelationMatrix = AllUsersRating.corr(method ='pearson',min_periods=MinUsersRating)
    SimilarMoviesCandidates = pd.Series([],dtype=pd.StringDtype()) 
    for i in range(0,len(NewUserRating.title)):
        #print("Adding Similarity For "+NewUserRating.title[i])
        MoviesSimilarties = MoviesCorrelationMatrix[NewUserRating.title[i]].dropna()
        MoviesSimilarties = MoviesSimilarties.map(lambda x:x*NewUserRating.rating[i])
        SimilarMoviesCandidates = SimilarMoviesCandidates.append(MoviesSimilarties)
    SimilarMoviesCandidates.sort_values(inplace=True,ascending=False)
    similarMoviesCandidates=SimilarMoviesCandidates.groupby(SimilarMoviesCandidates.index).sum()
    similarMoviesCandidates.sort_values(inplace=True,ascending=False)
    return similarMoviesCandidates.drop(NewUserRating.title,errors='ignore') 

In [7]:
import pandas as pd
RatingsColumns = ['userId','movieId','rating']
RatingsDataSet = pd.read_csv('ratings.dataset',sep='\t',names=RatingsColumns,usecols=range(3))
MoviesColumns = ['movieId','title']
MoviesDataSet = pd.read_csv('movies.dataset',sep='|',names=MoviesColumns,usecols=range(2))
MoviesRatingDataSet = pd.merge(MoviesDataSet,RatingsDataSet)

In [12]:
UserRatingData = getUserRatingData(userId=0,MoviesRatingDataSet=MoviesRatingDataSet)

In [10]:
UserRatingData.head()

,title,rating
0,Toy Story (1995),5
1,Star Wars (1977),5
2,Gone with the Wind (1939),1
3,"Empire Strikes Back, The (1980)",5


In [14]:
MoviesRecommendation = CFRecommender(MoviesRatingDataSet,userId=0)

In [15]:
MoviesRecommendation.head()

Return of the Jedi (1983)                    8.113633
Raiders of the Lost Ark (1981)               7.351770
Cinderella (1950)                            5.387272
Indiana Jones and the Last Crusade (1989)    5.252377
Wizard of Oz, The (1939)                     4.963759
dtype: float64